In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



# Load feature data (X)
#features_file_path = 'train.csv'
#all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
#selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]  # Replace with your actual feature column names
#selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
#X = selected_features
#y = all_features.iloc[:, -1]

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Load feature data (X)
features_file_path = 'train.csv'
all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]
selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
X = selected_features
y = all_features.iloc[:, -1]

# Handle class imbalance using oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# Create individual models with regularization and scaling
mlp_classifier = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, alpha=0.01, random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale', C=1.0))

# Create the ensemble model using stacking with logistic regression as the final estimator
ensemble_classifier = StackingClassifier(
    estimators=[
        ('neural_network', mlp_classifier),
        ('svm', svm_classifier),
    ],
    final_estimator=LogisticRegression()
)

# Train the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Evaluate the ensemble model on the test set
ensemble_predictions = ensemble_classifier.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

# Load test feature data
test_features_file_path = 'test.csv'
test_features = pd.read_csv(test_features_file_path)
# Specify the columns used as features in the test set
selected_test_features = test_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]

# Convert 'Yes' and 'No' to 1 and 0
selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Load test label data
test_labels_file_path = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file_path)
test_labels = test_labels.iloc[:, -1]


# Use the trained ensemble model to make predictions on the test set
test_predictions = ensemble_classifier.predict(selected_test_features)

# Print or save the predictions and actual labels for comparison
result_df = pd.DataFrame({'Actual Labels': test_labels, 'Predicted Labels': test_predictions})
print(result_df)

# Print instances where the predicted labels do not match the actual labels
mismatched_instances = result_df[result_df['Actual Labels'] != result_df['Predicted Labels']]
print("Mismatched Instances:")
print(mismatched_instances)

# Print overall comparison
print("\nComparison of Actual and Predicted Labels:")
print(result_df)

ensemble_accuracy = accuracy_score(test_labels, test_predictions)
ensemble_precision = precision_score(test_labels, test_predictions)
ensemble_recall = recall_score(test_labels, test_predictions)
ensemble_f1 = f1_score(test_labels, test_predictions)

# Print evaluation metrics
print("\nEnsemble Model Evaluation Metrics:")
print(f"Accuracy: {ensemble_accuracy}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1-Score: {ensemble_f1}")


<ipython-input-2-5f58428b40cb>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)


Ensemble Model Accuracy: 0.5884766159175859


<ipython-input-2-5f58428b40cb>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)


       Actual Labels  Predicted Labels
0                  1                 1
1                  1                 1
2                  1                 0
3                  1                 0
4                  1                 0
...              ...               ...
39058              1                 0
39059              1                 1
39060              1                 1
39061              1                 1
39062              1                 0

[39063 rows x 2 columns]
Mismatched Instances:
       Actual Labels  Predicted Labels
2                  1                 0
3                  1                 0
4                  1                 0
5                  1                 0
6                  1                 0
...              ...               ...
39055              1                 0
39056              1                 0
39057              1                 0
39058              1                 0
39062              1                 0

[23284 rows x 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler




# Load feature data (X)
#features_file_path = 'train.csv'
#all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
#selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]  # Replace with your actual feature column names
#selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
#X = selected_features
#y = all_features.iloc[:, -1]

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Load feature data (X)
features_file_path = 'train.csv'
all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]
selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
X = selected_features
y = all_features.iloc[:, -1]

# Handle class imbalance using oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# Create individual models with regularization and scaling
# mlp_classifier = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, alpha=0.01, random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale', C=1.0))

# Create individual models
gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Create the ensemble model using stacking with logistic regression as the final estimator
ensemble_classifier = StackingClassifier(
    estimators=[
        # Comment or remove this line
        ('svm', svm_classifier),
        ('gradient_boosting', gradient_boosting_classifier),
        # Add more models as needed
    ],
    final_estimator=LogisticRegression()
)

# Train the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Evaluate the ensemble model on the test set
ensemble_predictions = ensemble_classifier.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

# Load test feature data
test_features_file_path = 'test.csv'
test_features = pd.read_csv(test_features_file_path)
# Specify the columns used as features in the test set
selected_test_features = test_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]

# Convert 'Yes' and 'No' to 1 and 0
selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Load test label data
test_labels_file_path = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file_path)
test_labels = test_labels.iloc[:, -1]


# Use the trained ensemble model to make predictions on the test set
test_predictions = ensemble_classifier.predict(selected_test_features)

# Print or save the predictions and actual labels for comparison
result_df = pd.DataFrame({'Actual Labels': test_labels, 'Predicted Labels': test_predictions})
print(result_df)

# Print instances where the predicted labels do not match the actual labels
mismatched_instances = result_df[result_df['Actual Labels'] != result_df['Predicted Labels']]
print("Mismatched Instances:")
print(mismatched_instances)

# Print overall comparison
print("\nComparison of Actual and Predicted Labels:")
print(result_df)

ensemble_accuracy = accuracy_score(test_labels, test_predictions)
ensemble_precision = precision_score(test_labels, test_predictions)
ensemble_recall = recall_score(test_labels, test_predictions)
ensemble_f1 = f1_score(test_labels, test_predictions)

# Print evaluation metrics
print("\nEnsemble Model Evaluation Metrics:")
print(f"Accuracy: {ensemble_accuracy}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1-Score: {ensemble_f1}")

<ipython-input-1-b0e13947ec6e>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)


Ensemble Model Accuracy: 0.5913027623302033


<ipython-input-1-b0e13947ec6e>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)


       Actual Labels  Predicted Labels
0                  1                 1
1                  1                 1
2                  1                 0
3                  1                 0
4                  1                 0
...              ...               ...
39058              1                 0
39059              1                 1
39060              1                 1
39061              1                 1
39062              1                 1

[39063 rows x 2 columns]
Mismatched Instances:
       Actual Labels  Predicted Labels
2                  1                 0
3                  1                 0
4                  1                 0
5                  1                 0
6                  1                 0
...              ...               ...
39053              1                 0
39055              1                 0
39056              1                 0
39057              1                 0
39058              1                 0

[20805 rows x 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler




# Load feature data (X)
#features_file_path = 'train.csv'
#all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
#selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]  # Replace with your actual feature column names
#selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
#X = selected_features
#y = all_features.iloc[:, -1]

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Load feature data (X)
features_file_path = 'train.csv'
all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]
selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
X = selected_features
y = all_features.iloc[:, -1]

# Handle class imbalance using oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# Create individual models with regularization and scaling
# mlp_classifier = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, alpha=0.01, random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale', C=1.0))

# Create individual models
lgbm_classifier = LGBMClassifier(n_estimators=100, random_state=42)

# Create the ensemble model using stacking with logistic regression as the final estimator
ensemble_classifier = StackingClassifier(
    estimators=[
        # Comment or remove this line
        ('svm', svm_classifier),
        ('lgbm', lgbm_classifier),
        # Add more models as needed
    ],
    final_estimator=LogisticRegression()
)

# Train the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Evaluate the ensemble model on the test set
ensemble_predictions = ensemble_classifier.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, ensemble_predictions)

print("Test Set Results:")
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")
print("Confusion Matrix for Test Set:")
print(conf_matrix_test)

# Load test feature data
test_features_file_path = 'test.csv'
test_features = pd.read_csv(test_features_file_path)
# Specify the columns used as features in the test set
selected_test_features = test_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]

# Convert 'Yes' and 'No' to 1 and 0
selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Load test label data
test_labels_file_path = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file_path)
test_labels = test_labels.iloc[:, -1]


# Use the trained ensemble model to make predictions on the test set
test_predictions = ensemble_classifier.predict(selected_test_features)

# Print or save the predictions and actual labels for comparison
result_df = pd.DataFrame({'Actual Labels': test_labels, 'Predicted Labels': test_predictions})
print(result_df)

# Print instances where the predicted labels do not match the actual labels
mismatched_instances = result_df[result_df['Actual Labels'] != result_df['Predicted Labels']]
print("Mismatched Instances:")
print(mismatched_instances)

# Print overall comparison
print("\nComparison of Actual and Predicted Labels:")
print(result_df)

ensemble_accuracy = accuracy_score(test_labels, test_predictions)
ensemble_precision = precision_score(test_labels, test_predictions)
ensemble_recall = recall_score(test_labels, test_predictions)
ensemble_f1 = f1_score(test_labels, test_predictions)

# Confusion Matrix for Training Set
conf_matrix_train = confusion_matrix(test_labels, test_predictions)

# Print evaluation metrics
print("\nEnsemble Model Evaluation Metrics:")
print(f"Accuracy: {ensemble_accuracy}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1-Score: {ensemble_f1}")
print("Confusion Matrix for Training Set:")
print(conf_matrix_train)

<ipython-input-2-dfceb293a380>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)


[LightGBM] [Info] Number of positive: 43998, number of negative: 43752
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 131
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501402 -> initscore=0.005607
[LightGBM] [Info] Start training from score 0.005607
[LightGBM] [Info] Number of positive: 35198, number of negative: 35002
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130
[LightGBM] [Info] Number of data points in the train set: 70200, number of used features: 20
[LightGBM] [Info] [bin

<ipython-input-2-dfceb293a380>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)


       Actual Labels  Predicted Labels
0                  1                 1
1                  1                 1
2                  1                 0
3                  1                 0
4                  1                 0
...              ...               ...
39058              1                 1
39059              1                 1
39060              1                 0
39061              1                 1
39062              1                 0

[39063 rows x 2 columns]
Mismatched Instances:
       Actual Labels  Predicted Labels
2                  1                 0
3                  1                 0
4                  1                 0
5                  1                 0
6                  1                 0
...              ...               ...
39052              1                 0
39055              1                 0
39057              1                 0
39060              1                 0
39062              1                 0

[21110 rows x 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler




# Load feature data (X)
#features_file_path = 'train.csv'
#all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
#selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]  # Replace with your actual feature column names
#selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
#X = selected_features
#y = all_features.iloc[:, -1]

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Load feature data (X)
features_file_path = 'train.csv'
all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]
selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
X = selected_features
y = all_features.iloc[:, -1]

# Handle class imbalance using oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# Create individual models with regularization and scaling
# mlp_classifier = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, alpha=0.01, random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale', C=1.0))

# Create individual models
xgb_classifier = XGBClassifier()

# Create the ensemble model using stacking with logistic regression as the final estimator
ensemble_classifier = StackingClassifier(
    estimators=[
        # Comment or remove this line
        ('svm', svm_classifier),
        ('xgboost', xgb_classifier),
        # Add more models as needed
    ],
    final_estimator=LogisticRegression()
)

# Train the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Evaluate the ensemble model on the test set
ensemble_predictions = ensemble_classifier.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, ensemble_predictions)

print("Test Set Results:")
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")
print("Confusion Matrix for Test Set:")
print(conf_matrix_test)

# Load test feature data
test_features_file_path = 'test.csv'
test_features = pd.read_csv(test_features_file_path)
# Specify the columns used as features in the test set
selected_test_features = test_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]

# Convert 'Yes' and 'No' to 1 and 0
selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Load test label data
test_labels_file_path = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file_path)
test_labels = test_labels.iloc[:, -1]


# Use the trained ensemble model to make predictions on the test set
test_predictions = ensemble_classifier.predict(selected_test_features)

# Print or save the predictions and actual labels for comparison
result_df = pd.DataFrame({'Actual Labels': test_labels, 'Predicted Labels': test_predictions})
print(result_df)

# Print instances where the predicted labels do not match the actual labels
mismatched_instances = result_df[result_df['Actual Labels'] != result_df['Predicted Labels']]
print("Mismatched Instances:")
print(mismatched_instances)

# Print overall comparison
print("\nComparison of Actual and Predicted Labels:")
print(result_df)

ensemble_accuracy = accuracy_score(test_labels, test_predictions)
ensemble_precision = precision_score(test_labels, test_predictions)
ensemble_recall = recall_score(test_labels, test_predictions)
ensemble_f1 = f1_score(test_labels, test_predictions)

# Confusion Matrix for Training Set
conf_matrix_train = confusion_matrix(test_labels, test_predictions)

# Print evaluation metrics
print("\nEnsemble Model Evaluation Metrics:")
print(f"Accuracy: {ensemble_accuracy}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1-Score: {ensemble_f1}")
print("Confusion Matrix for Training Set:")
print(conf_matrix_train)

<ipython-input-3-a74c7365773b>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)


Test Set Results:
Ensemble Model Accuracy: 0.6434953049503145
Confusion Matrix for Test Set:
[[6543 4549]
 [3272 7574]]


<ipython-input-3-a74c7365773b>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)


       Actual Labels  Predicted Labels
0                  1                 1
1                  1                 0
2                  1                 0
3                  1                 0
4                  1                 0
...              ...               ...
39058              1                 0
39059              1                 1
39060              1                 0
39061              1                 1
39062              1                 0

[39063 rows x 2 columns]
Mismatched Instances:
       Actual Labels  Predicted Labels
1                  1                 0
2                  1                 0
3                  1                 0
4                  1                 0
5                  1                 0
...              ...               ...
39052              1                 0
39055              1                 0
39058              1                 0
39060              1                 0
39062              1                 0

[22556 rows x 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler




# Load feature data (X)
#features_file_path = 'train.csv'
#all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
#selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]  # Replace with your actual feature column names
#selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
#X = selected_features
#y = all_features.iloc[:, -1]

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Load feature data (X)
features_file_path = 'train.csv'
all_features = pd.read_csv(features_file_path)

# Specify the columns you want to use as features
selected_features = all_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]
selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Assuming the last column is the target variable (label)
X = selected_features
y = all_features.iloc[:, -1]

# Handle class imbalance using oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


# Create individual models with regularization and scaling
# mlp_classifier = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, alpha=0.01, random_state=42))
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale', C=1.0))

# Create individual models
adaboost_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)

# Create the ensemble model using stacking with logistic regression as the final estimator
ensemble_classifier = StackingClassifier(
    estimators=[
        # Comment or remove this line
        ('svm', svm_classifier),
        ('adaboost', adaboost_classifier),
        # Add more models as needed
    ],
    final_estimator=LogisticRegression()
)

# Train the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Evaluate the ensemble model on the test set
ensemble_predictions = ensemble_classifier.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, ensemble_predictions)

print("Test Set Results:")
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")
print("Confusion Matrix for Test Set:")
print(conf_matrix_test)

# Load test feature data
test_features_file_path = 'test.csv'
test_features = pd.read_csv(test_features_file_path)
# Specify the columns used as features in the test set
selected_test_features = test_features[['age_of_car', 'population_density', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'gross_weight', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']]

# Convert 'Yes' and 'No' to 1 and 0
selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)

# Load test label data
test_labels_file_path = 'sample_submission.csv'
test_labels = pd.read_csv(test_labels_file_path)
test_labels = test_labels.iloc[:, -1]


# Use the trained ensemble model to make predictions on the test set
test_predictions = ensemble_classifier.predict(selected_test_features)

# Print or save the predictions and actual labels for comparison
result_df = pd.DataFrame({'Actual Labels': test_labels, 'Predicted Labels': test_predictions})
print(result_df)

# Print instances where the predicted labels do not match the actual labels
mismatched_instances = result_df[result_df['Actual Labels'] != result_df['Predicted Labels']]
print("Mismatched Instances:")
print(mismatched_instances)

# Print overall comparison
print("\nComparison of Actual and Predicted Labels:")
print(result_df)

ensemble_accuracy = accuracy_score(test_labels, test_predictions)
ensemble_precision = precision_score(test_labels, test_predictions)
ensemble_recall = recall_score(test_labels, test_predictions)
ensemble_f1 = f1_score(test_labels, test_predictions)

# Confusion Matrix for Training Set
conf_matrix_train = confusion_matrix(test_labels, test_predictions)

# Print evaluation metrics
print("\nEnsemble Model Evaluation Metrics:")
print(f"Accuracy: {ensemble_accuracy}")
print(f"Precision: {ensemble_precision}")
print(f"Recall: {ensemble_recall}")
print(f"F1-Score: {ensemble_f1}")
print("Confusion Matrix for Training Set:")
print(conf_matrix_train)

<ipython-input-5-91c5bbe2820b>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features.replace({'Yes': 1, 'No': 0}, inplace=True)


Test Set Results:
Ensemble Model Accuracy: 0.5562494302124168
Confusion Matrix for Test Set:
[[5024 6068]
 [3667 7179]]


<ipython-input-5-91c5bbe2820b>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_test_features.replace({'Yes': 1, 'No': 0}, inplace=True)


       Actual Labels  Predicted Labels
0                  1                 1
1                  1                 1
2                  1                 0
3                  1                 1
4                  1                 0
...              ...               ...
39058              1                 0
39059              1                 1
39060              1                 0
39061              1                 1
39062              1                 0

[39063 rows x 2 columns]
Mismatched Instances:
       Actual Labels  Predicted Labels
2                  1                 0
4                  1                 0
5                  1                 0
6                  1                 0
8                  1                 0
...              ...               ...
39056              1                 0
39057              1                 0
39058              1                 0
39060              1                 0
39062              1                 0

[17435 rows x 2